In [1]:
import pandas as pd
import numpy as np 
import spacy

In [2]:
import random

In [3]:
from spacy.util import minibatch, compounding

In [4]:
from spacy.training import Example

In [5]:
import os

In [6]:
import ast

In [7]:
import re

In [8]:
cust_label = pd.read_csv("C:/Users/shrey/OneDrive/Desktop/TY/CCNLP/mini-project/train.csv")
cust_label

,source,ingredient_id,token_id,token,label
0,ar,0,0,4,QUANTITY
1,ar,0,1,cloves,UNIT
2,ar,0,2,garlic,NAME
3,ar,1,0,2,QUANTITY
4,ar,1,1,tablespoons,UNIT
...,...,...,...,...,...
37235,gk,5141,21,the,O
37236,gk,5141,22,larger,O
37237,gk,5141,23,amount,O
37238,gk,5141,24,o,O


In [9]:
ing_list = []
temp = cust_label[cust_label.label == 'NAME']
for i in temp['token']:
    ing_list.append(i)

In [10]:
ing_list

['garlic',
 'vegetable',
 'oil',
 'marjoram',
 'red',
 'onion',
 'jalapeno',
 'peppers',
 'olive',
 'oil',
 'extra',
 'virgin',
 'olive',
 'oil',
 'olives',
 'garlic',
 'oyster',
 'sauce',
 'tomato',
 'soup',
 'white',
 'rice',
 'chili',
 'bean',
 'paste',
 'white',
 'vinegar',
 'ginger',
 'root',
 'limes',
 'cremini',
 'mushrooms',
 'zucchini',
 'olive',
 'oil',
 'meat',
 'juice',
 'ginger',
 'water',
 'flour',
 'tortillas',
 'cooking',
 'spray',
 'onion',
 'chicken',
 'stock',
 'Greek-style',
 'yogurt',
 'cumin',
 'seeds',
 'cilantro',
 'zucchini',
 'milk',
 'red',
 'onion',
 'cream',
 'cheese',
 'butter',
 'chestnut',
 'puree',
 'chili',
 'powder',
 'honey',
 'salt',
 'ginger',
 'root',
 'curry',
 'powder',
 'shrimp',
 'green',
 'onions',
 'red',
 'pepper',
 'flakes',
 'olive',
 'oil',
 'salt',
 'chicken',
 'breast',
 'halves',
 'white',
 'wine',
 'Swiss',
 'cheese',
 'queijo',
 'coalho',
 'cheese',
 'curds',
 'egg',
 'yolk',
 'milk',
 'Cheddar',
 'cheese',
 'white',
 'sugar',
 'hal

In [11]:
cust_label.dropna(axis=0, inplace=True)

In [12]:
cust_label[['token', 'label']].isna().sum()

token    0
label    0
dtype: int64

In [13]:
ing = np.array(cust_label['token'])
label = np.array(cust_label['label'])
label

array(['QUANTITY', 'UNIT', 'NAME', ..., 'O', 'O', 'O'], dtype=object)

In [14]:
for i in ing:
    if i == 'minute':
        print(i)

In [15]:
for i in ing:
    print(i)

4
cloves
garlic
2
tablespoons
vegetable
oil
,
divided
2
tablespoons
dried
marjoram
1
large
red
onion
,
1/4-inch
slices
pulled
into
rings
2
jalapeno
peppers
,
seeded
and
minced
1
tablespoon
olive
oil
extra
virgin
olive
oil
,
for
drizzling
1/3
cup
sliced
pitted
ripe
olives
-LRB-
optional
-RRB-
3
cloves
garlic
,
minced
4
tablespoons
oyster
sauce
1
-LRB-
10.75
ounce
-RRB-
can
condensed
tomato
soup
2
cups
uncooked
white
rice
2
tablespoons
chili
bean
paste
-LRB-
toban
djan
-RRB-
3/4
cup
distilled
white
vinegar
1
teaspoon
grated
fresh
ginger
root
2
limes
,
sliced
into
wedges
1
cup
cremini
mushrooms
,
cut
in
half
1
zucchini
,
halved
lengthwise
and
cut
into
1
inch
slices
1/2
cup
olive
oil
,
or
to
taste
4
12-inch
long
metal
skewers
1
cup
canned
lobster
meat
,
with
juice
2
tablespoons
drained
pickled
ginger
,
for
garnish
4
cups
water
8
-LRB-
8
inch
-RRB-
flour
tortillas
cooking
spray
1
large
onion
,
chopped
6
cups
chicken
stock
,
divided
1
tablespoon
fat-free
Greek-style
yogurt
1
teaspoon
cumin
s

In [16]:
train_data = []
for i,j in enumerate(ing):
    print(j, label[i], len(j))
    train_data.append((j, {"entities" : [(0, len(j), label[i])]}))

4 QUANTITY 1
cloves UNIT 6
garlic NAME 6
2 QUANTITY 1
tablespoons UNIT 11
vegetable NAME 9
oil NAME 3
, O 1
divided STATE 7
2 QUANTITY 1
tablespoons UNIT 11
dried DF 5
marjoram NAME 8
1 QUANTITY 1
large SIZE 5
red NAME 3
onion NAME 5
, O 1
1/4-inch O 8
slices O 6
pulled O 6
into O 4
rings O 5
2 QUANTITY 1
jalapeno NAME 8
peppers NAME 7
, O 1
seeded STATE 6
and O 3
minced STATE 6
1 QUANTITY 1
tablespoon UNIT 10
olive NAME 5
oil NAME 3
extra NAME 5
virgin NAME 6
olive NAME 5
oil NAME 3
, O 1
for O 3
drizzling O 9
1/3 QUANTITY 3
cup UNIT 3
sliced STATE 6
pitted STATE 6
ripe STATE 4
olives NAME 6
-LRB- O 5
optional O 8
-RRB- O 5
3 QUANTITY 1
cloves UNIT 6
garlic NAME 6
, O 1
minced STATE 6
4 QUANTITY 1
tablespoons UNIT 11
oyster NAME 6
sauce NAME 5
1 QUANTITY 1
-LRB- O 5
10.75 QUANTITY 5
ounce UNIT 5
-RRB- O 5
can UNIT 3
condensed STATE 9
tomato NAME 6
soup NAME 4
2 QUANTITY 1
cups UNIT 4
uncooked STATE 8
white NAME 5
rice NAME 4
2 QUANTITY 1
tablespoons UNIT 11
chili NAME 5
bean NAME 4
pa

 NAME 5
1/2 QUANTITY 3
cup UNIT 3
caster NAME 6
sugar NAME 5
40 QUANTITY 2
g UNIT 1
butter NAME 6
1 QUANTITY 1
package UNIT 7
phyllo NAME 6
pastry NAME 6
1 QUANTITY 1
ounce UNIT 5
grapefruit NAME 10
juice NAME 5
3/4 QUANTITY 3
ounce UNIT 5
amaretto NAME 8
almond NAME 6
liqueur NAME 7
1 QUANTITY 1
can UNIT 3
mushroom NAME 8
soup NAME 4
1 QUANTITY 1
tablespoon UNIT 10
ketchup NAME 7
1/4 QUANTITY 3
cup UNIT 3
sauvignon NAME 9
blanc NAME 5
wine NAME 4
1/2 QUANTITY 3
tablespoon UNIT 10
apple NAME 5
cider NAME 5
vinegar NAME 7
3 QUANTITY 1
g UNIT 1
salt NAME 4
1 QUANTITY 1
head UNIT 4
broccoli NAME 8
1/2 QUANTITY 3
teaspoon UNIT 8
allspice NAME 8
, O 1
ground STATE 6
1 QUANTITY 1
teaspoon UNIT 8
lemon NAME 5
pepper NAME 6
1 QUANTITY 1
lb UNIT 2
mushroom NAME 8
1/4 QUANTITY 3
cup UNIT 3
milk NAME 4
1 QUANTITY 1
lime NAME 4
660 QUANTITY 3
g UNIT 1
silken-soft STATE 11
tofu NAME 4
1 QUANTITY 1
teaspoon UNIT 8
caster NAME 6
sugar NAME 5
1 QUANTITY 1
teaspoon UNIT 8
coconut NAME 7
extract NAME 7


In [17]:
train_data

[('4', {'entities': [(0, 1, 'QUANTITY')]}),
 ('cloves', {'entities': [(0, 6, 'UNIT')]}),
 ('garlic', {'entities': [(0, 6, 'NAME')]}),
 ('2', {'entities': [(0, 1, 'QUANTITY')]}),
 ('tablespoons', {'entities': [(0, 11, 'UNIT')]}),
 ('vegetable', {'entities': [(0, 9, 'NAME')]}),
 ('oil', {'entities': [(0, 3, 'NAME')]}),
 (',', {'entities': [(0, 1, 'O')]}),
 ('divided', {'entities': [(0, 7, 'STATE')]}),
 ('2', {'entities': [(0, 1, 'QUANTITY')]}),
 ('tablespoons', {'entities': [(0, 11, 'UNIT')]}),
 ('dried', {'entities': [(0, 5, 'DF')]}),
 ('marjoram', {'entities': [(0, 8, 'NAME')]}),
 ('1', {'entities': [(0, 1, 'QUANTITY')]}),
 ('large', {'entities': [(0, 5, 'SIZE')]}),
 ('red', {'entities': [(0, 3, 'NAME')]}),
 ('onion', {'entities': [(0, 5, 'NAME')]}),
 (',', {'entities': [(0, 1, 'O')]}),
 ('1/4-inch', {'entities': [(0, 8, 'O')]}),
 ('slices', {'entities': [(0, 6, 'O')]}),
 ('pulled', {'entities': [(0, 6, 'O')]}),
 ('into', {'entities': [(0, 4, 'O')]}),
 ('rings', {'entities': [(0, 5, 'O

In [18]:
# # Setting up the pipeline and entity recognizer.
# model = None
# if model is not None:
#     nlp = spacy.load(model)  # load existing spacy model
#     print("Loaded model '%s'" % model)
# else:
#     nlp = spacy.blank('en')  # create blank Language class
#     print("Created blank 'en' model")

# print(nlp.pipe_names)

# if 'ner' not in nlp.pipe_names:
#     ner = nlp.create_pipe('ner')
#     nlp.add_pipe('ner')
# else:
#     ner = nlp.get_pipe('ner')

In [19]:
# # Add new entity labels to entity recognizer
# for i in cust_label['label'].unique():
#     ner.add_label(i)# Inititalizing optimizer
# if model is None:
#     optimizer = nlp.begin_training()
# else:
#     optimizer = nlp.entity.create_optimizer()

In [20]:
# # Get names of other pipes to disable them during training to train # only NER and update the weights
# n_iter = 2
# other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
# with nlp.disable_pipes(*other_pipes):  # only train NER
#     for itn in range(n_iter):
#         random.shuffle(train_data)
#         losses = {}
#         batches = minibatch(train_data, 
#                             size=compounding(4., 32., 1.001))
#         #for batch in batches:
#         #    texts, annotations = zip(*batch) 
#         #    #print(texts, annotations)
#         #    # create Example
#         #    print(texts)
#         #    doc = nlp.make_doc(texts)
#         #    example = Example.from_dict(doc, annotations)
#         #    # Updating the weights
#         #    nlp.update([example], sgd=optimizer, 
#         #               drop=0.35, losses=losses)
#         #print('Losses', losses)       
        
        
#         for batch in batches:
#             texts, annotations = zip(*batch)
    
#             example = []
#             # Update the model with iterating each text
#             for i in range(len(texts)):
#                 doc = nlp.make_doc(texts[i])
#                 print(annotations[i])
#                 example.append(Example.from_dict(doc, annotations[i]))
    
#             # Update the model
#             nlp.update(example, drop=0.35, losses=losses, sgd=optimizer)
#         print('Losses', losses)

In [21]:
# # Save model 
# output_dir = "C:/Users/shubh/CCNLP"
# nlp.to_disk(output_dir)
# print("Saved model to", output_dir)

In [22]:
# Test the saved model
output_dir = "C:/Users/shrey/OneDrive/Desktop/TY/CCNLP/mini-project"
test_text = "I want something spicy made from chicken, brocolli, tomato and potato"
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
doc2 = nlp2(test_text)
for ent in doc2.ents:
    print(ent.label_, ent.text)

Loading from C:/Users/shrey/OneDrive/Desktop/TY/CCNLP/mini-project
O I
UNIT want
O something
NAME made from chicken
O ,
NAME brocolli
NAME tomato
NAME potato


In [23]:
doc2 = nlp2("I'm craving a delicious recipe that features ripe avocados")
for ent in doc2.ents:
    print(ent.label_, ent.text)

O I
NAME craving a delicious
NAME features
STATE ripe
NAME avocados


In [24]:
doc2 = nlp2("I have a bunch of fresh tomato, basil, and mozzarella at home Can you recommend a tasty recipe that includes these ingredients")
for ent in doc2.ents:
    print(ent.label_, ent.text)

O I
O of
DF fresh
NAME tomato
NAME basil
O , and
NAME mozzarella
O Can you
O ingredients


In [25]:
doc2 = nlp2("I have one bacon, two tomatoes, 3 onions and one packet of cheese. what can I make?")
for ent in doc2.ents:
    print(ent.label_, ent.text)

O I
NAME bacon
O ,
NAME tomatoes
O ,
QUANTITY 3
NAME onions and one packet
O of
NAME cheese
UNIT can
O I
O ?


In [26]:
doc2 = nlp2("I want to make something within 30 minute. what can I make?")
for ent in doc2.ents:
    print(ent.label_, ent.text)

O I
O to
O within
QUANTITY 30
STATE minute. what
UNIT can
O I
O ?


In [27]:
doc2 = nlp2("I want to make something within 20 minute. what can I make?")
for ent in doc2.ents:
    print(ent.label_, ent.text)

O I
O to
O within
QUANTITY 20
UNIT can
O I
O ?


In [28]:
ing_list = []
temp = cust_label[cust_label.label == 'NAME']
for i in temp['token']:
    ing_list.append(i)
# ing_list

In [29]:
recipes = pd.read_csv('C:/Users/shrey/OneDrive/Desktop/TY/CCNLP/mini-project/RAW_recipes.csv')
recipes.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [30]:
recipes.isna().sum()

name                 1
id                   0
minutes              0
contributor_id       0
submitted            0
tags                 0
nutrition            0
n_steps              0
steps                0
description       4979
ingredients          0
n_ingredients        0
dtype: int64

In [31]:
recipes.shape

(231637, 12)

In [32]:
recipes.dropna(axis=0, inplace=True)

In [33]:
recipes.shape

(226657, 12)

In [34]:
recipes.isna().sum()

name              0
id                0
minutes           0
contributor_id    0
submitted         0
tags              0
nutrition         0
n_steps           0
steps             0
description       0
ingredients       0
n_ingredients     0
dtype: int64

In [35]:
ingredients_compare = []
for i in recipes.ingredients:
    ingredients_compare.append(ast.literal_eval(i))

In [36]:
recipes['ingredients_compare'] = ingredients_compare

In [37]:
steps_list = []
for i in recipes.steps:
    steps_list.append(ast.literal_eval(i))
#print(steps_list[0])

In [38]:
recipes['steps_new'] = steps_list

In [39]:
recipes.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,ingredients_compare,steps_new
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,"[winter squash, mexican seasoning, mixed spice...","[make a choice and proceed with recipe, depend..."
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,"[prepared pizza crust, sausage patty, eggs, mi...","[preheat oven to 425 degrees f, press dough in..."
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,"[ground beef, yellow onions, diced tomatoes, t...","[brown ground beef in large pot, add chopped o..."
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,"[spreadable cheese with garlic and herbs, new ...",[place potatoes in a large pot of lightly salt...
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,"[tomato juice, apple cider vinegar, sugar, sal...","[mix all ingredients& boil for 2 1 / 2 hours ,..."


In [40]:
recipes['steps_new'][10000][1]

'place in a steamer basket and steam until just tender , about 5 minutes'

In [41]:
# Function to calculate Jaccard similarity
def jaccard_similarity(list1, list2):
    intersection = len(set(list1) & set(list2))
    union = len(set(list1) | set(list2))
    return intersection / union if union != 0 else 0

In [42]:
def get_ing():
    l1 =[]
    in_str = input("What ingredients do you have?")
    doc = nlp2(in_str)
    for ent in doc.ents:
        if ent.label_ == 'NAME':
            temp = ent.text.split()
            for i in temp:
                if i in ing_list:
                    l1.append(i)
    
    return l1

def get_recipes():
    temp = get_ing()
    # Use apply along with a lambda function to compare each element in the lists
    recipes['Match'] = recipes['ingredients'].apply(lambda x: all(item in x for item in temp))
    temp2 = recipes[recipes['Match']==True]
    #print(len(temp2))
    if len(temp2) !=0:   
        temp2['Similarity'] = temp2['ingredients_compare'].apply(lambda x: jaccard_similarity(x, temp))
        # Find the row with the highest similarity
        best_match_row = temp2.loc[temp2['Similarity'].idxmax()]
        top_5_matches = temp2.nlargest(5, 'Similarity')
        top_5_matches.reset_index()
    else: # This case runs when theres no complete match of ingredients
        recipes['Similarity'] = recipes['ingredients_compare'].apply(lambda x: jaccard_similarity(x, temp))
        # Find the row with the highest similarity
        best_match_row = recipes.loc[recipes['Similarity'].idxmax()]
        top_5_matches = recipes.nlargest(5, 'Similarity')
        top_5_matches.reset_index()

    # Print the DataFrame with the similarity scores
    #print("\nBest Matching List:")
    #print(best_match_row['ingredients'])
    #print("Jaccard Similarity:", best_match_row['Similarity'])
    #print(temp)
    #print("Top 5: ",top_5_matches[['ingredients', 'Similarity']])
    ascii_art = """
 ######  #######  #####  #######    ######  #######  #####  ### ####### ######  ####### 
#     # #       #     #    #       #     # #       #     #  #  #       #     # #       
#     # #       #          #       #     # #       #        #  #       #     # #       
######  #####    #####     #       ######  #####   #        #  #####   ######  #####   
#     # #             #    #       #   #   #       #        #  #       #       #       
#     # #       #     #    #       #    #  #       #     #  #  #       #       #       
######  #######  #####     #       #     # #######  #####  ### ####### #       ####### 
"""

    print(ascii_art)
    
    print("Here are some of the recipes that might be relevant for you: ")
    #print(top_5_matches.iloc[1])
    for i in range(5):
        t = top_5_matches.iloc[i]
        print('Name of dish',i+1,' : ', recipes['name'][t.name])
        print(f'Ingredients required: {t.ingredients}')
        #print(t.steps)
        c = 1
        for i in t.steps_new:
            print(f'step {c}: {i}')
            c+=1
        print("\n\n")

In [43]:
get_recipes()


 ######  #######  #####  #######    ######  #######  #####  ### ####### ######  ####### 
#     # #       #     #    #       #     # #       #     #  #  #       #     # #       
#     # #       #          #       #     # #       #        #  #       #     # #       
######  #####    #####     #       ######  #####   #        #  #####   ######  #####   
#     # #             #    #       #   #   #       #        #  #       #       #       
#     # #       #     #    #       #    #  #       #     #  #  #       #       #       
######  #######  #####     #       #     # #######  #####  ### ####### #       ####### 

Here are some of the recipes that might be relevant for you: 
Name of dish 1  :  chicken dip
Ingredients required: ['chicken', 'cream cheese', 'onion', 'lime, juice of']
step 1: mix all together
step 2: refrigerate
step 3: serve with crackers
step 4: enjoy !



Name of dish 2  :  easy  tasty barley casserole
Ingredients required: ['pearl barley', 'onion', 'chicken', 'water']
ste

C:\Users\shrey\AppData\Local\Temp\ipykernel_8016\2379336549.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp2['Similarity'] = temp2['ingredients_compare'].apply(lambda x: jaccard_similarity(x, temp))
